In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import os
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping


In [10]:
dataset_crop_weeds = 'agri_data/data/'  # Update this path as necessary

# Images Sizes
img_height = 512
img_width = 512

# Defines batch size and number of epochs
batch_size = 32
epochs = 10


In [11]:
images = []
annotations = []

for filename in os.listdir(dataset_crop_weeds):
    if filename.endswith('.jpeg'):
        image_path = os.path.join(dataset_crop_weeds, filename)
        images.append(image_path)
    elif filename.endswith('.txt'):
        annotation_path = os.path.join(dataset_crop_weeds, filename)
        annotations.append(annotation_path)

print(f"Total Images: {len(images)}")
print(f"Total Annotations: {len(annotations)}")


Total Images: 1300
Total Annotations: 1300


In [12]:
train_images, test_images, train_annotations, test_annotations = train_test_split(images, annotations, test_size=0.2, random_state=42)
train_images, val_images, train_annotations, val_annotations = train_test_split(train_images, train_annotations, test_size=0.1, random_state=42)


In [13]:
def load_and_preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    image = cv2.resize(image, (img_width, img_height))
    return image / 255.0  # Normalize pixel values to [0, 1]

def parse_annotation(annotation_path):
    with open(annotation_path, 'r') as file:
        first_line = file.readline().strip()
        class_label = int(first_line.split()[0])  # Extracts the class label
    return class_label


In [14]:
train_images_data = np.array([load_and_preprocess_image(img) for img in train_images])
val_images_data = np.array([load_and_preprocess_image(img) for img in val_images])
test_images_data = np.array([load_and_preprocess_image(img) for img in test_images])

train_annotations_data = np.array([parse_annotation(ann) for ann in train_annotations])
val_annotations_data = np.array([parse_annotation(ann) for ann in val_annotations])
test_annotations_data = np.array([parse_annotation(ann) for ann in test_annotations])


In [15]:
data_gen_args = dict(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_datagen = ImageDataGenerator(**data_gen_args)


In [16]:
model = tf.keras.Sequential([
    InputLayer(input_shape=(img_height, img_width, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


In [17]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [18]:
history = model.fit(
    train_datagen.flow(train_images_data, train_annotations_data, batch_size=batch_size),
    epochs=epochs,
    validation_data=(val_images_data, val_annotations_data),
    callbacks=[early_stopping]
)


Epoch 1/10
30/30 [==============================] - 109s 4s/step - loss: 2.4388 - accuracy: 0.4754 - val_loss: 0.6932 - val_accuracy: 0.4712
Epoch 2/10
30/30 [==============================] - 95s 3s/step - loss: 0.6932 - accuracy: 0.5011 - val_loss: 0.6931 - val_accuracy: 0.5288
Epoch 3/10
30/30 [==============================] - 106s 3s/step - loss: 0.6931 - accuracy: 0.5128 - val_loss: 0.6930 - val_accuracy: 0.5288
Epoch 4/10
30/30 [==============================] - 102s 3s/step - loss: 0.6931 - accuracy: 0.5118 - val_loss: 0.6929 - val_accuracy: 0.5288
Epoch 5/10
30/30 [==============================] - 96s 3s/step - loss: 0.6930 - accuracy: 0.5118 - val_loss: 0.6928 - val_accuracy: 0.5288
Epoch 6/10
30/30 [==============================] - 102s 3s/step - loss: 0.6931 - accuracy: 0.5118 - val_loss: 0.6928 - val_accuracy: 0.5288
Epoch 7/10
30/30 [==============================] - 89s 3s/step - loss: 0.6931 - accuracy: 0.5118 - val_loss: 0.6928 - val_accuracy: 0.5288
Epoch 8/10
30/30

In [19]:
test_loss, test_acc = model.evaluate(test_images_data, test_annotations_data, verbose=2)
print(f"Test accuracy: {test_acc}")


9/9 - 7s - loss: 0.6930 - accuracy: 0.5077 - 7s/epoch - 779ms/step
Test accuracy: 0.5076923370361328
